In [2]:
from typing import Optional, Dict, List, Tuple
import json
import pandas as pd
import requests
from sqlmodel import SQLModel, create_engine, Field, Session, select
from pydantic import validator

pd.set_option('display.max_rows', 1000)

In [3]:
class BasicModel(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str
    # _clean_name = validator('name', pre=True, allow_reuse=True)(braces_validator)
    age: int
    interests: str
    # _interests: validator('interests', pre=True, allow_reuse=True)(interests_validator)
    
@validator('name')
def braces_validator(input_str: str) -> str:
    if any([braces in input_str for braces in ['{', '}']]):
        raise ValueError('Must contain a brace character. Example : { or }')
    braces = ['{', '}']
    for brace in braces:
        input_str = input_str.replace(brace, '')
    return input_str

@validator('interests')
def interests_validator(input_str: str) -> str:
    str_list = [s.strip() for s in input_str.split(',')]
    return json.dumps(str_list)

In [4]:
sqlite_file_name = "database.db"
sqlite_url = f"sqlite:///{sqlite_file_name}"
engine = create_engine(sqlite_url, echo=True)

def create_db_and_tables():
    SQLModel.metadata.create_all(engine)
create_db_and_tables()

2023-03-17 22:41:02,123 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-17 22:41:02,124 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("basicmodel")
2023-03-17 22:41:02,124 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-17 22:41:02,125 INFO sqlalchemy.engine.Engine COMMIT


In [5]:
obj_1 = BasicModel(
    name = '{Rick}',
    age = 34,
    interests = 'hockey, football, pole dancing'
)
obj_1

BasicModel(id=None, name='{Rick}', age=34, interests='hockey, football, pole dancing')

In [6]:
with Session(engine) as sess:
    sess.add(obj_1)
    sess.commit()

2023-03-17 22:41:04,749 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-17 22:41:04,751 INFO sqlalchemy.engine.Engine INSERT INTO basicmodel (name, age, interests) VALUES (?, ?, ?)
2023-03-17 22:41:04,751 INFO sqlalchemy.engine.Engine [generated in 0.00059s] ('{Rick}', 34, 'hockey, football, pole dancing')
2023-03-17 22:41:04,753 INFO sqlalchemy.engine.Engine COMMIT


In [7]:
with Session(engine) as session:
    statement = select(BasicModel)
    results = session.exec(statement) 
    for record in results:
        print(record)

2023-03-17 22:41:07,098 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-17 22:41:07,100 INFO sqlalchemy.engine.Engine SELECT basicmodel.id, basicmodel.name, basicmodel.age, basicmodel.interests 
FROM basicmodel
2023-03-17 22:41:07,101 INFO sqlalchemy.engine.Engine [generated in 0.00072s] ()
interests='hockey, football, pole dancing' id=1 name='{Rick}' age=34
2023-03-17 22:41:07,102 INFO sqlalchemy.engine.Engine ROLLBACK


In [15]:
my_dict = {
    'name' : 'Rick2',
    'age' : 35,
    'gender' : 'm'
}

In [18]:
obj_3 = BasicModel(**my_dict)
obj_3

BasicModel(id=None, name='Rick2', age=35)

In [79]:
obj_3

BasicModel()

In [19]:
with Session(engine) as sess:
    sess.add(obj_3)
    sess.commit()

2023-03-16 13:55:51,020 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-16 13:55:51,021 INFO sqlalchemy.engine.Engine INSERT INTO basicmodel (name, age) VALUES (?, ?)
2023-03-16 13:55:51,022 INFO sqlalchemy.engine.Engine [cached since 201.2s ago] ('Rick2', 35)
2023-03-16 13:55:51,023 INFO sqlalchemy.engine.Engine COMMIT
